In [1]:
# ===================================================================
# :일: 단계: 라이브러리 설치 및 임포트
# ===================================================================
!pip install xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,f1_score
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import warnings
import contextlib
import io
warnings.filterwarnings('ignore')
# ===================================================================
# :둘: 단계: 데이터 로드 및 전처리
# ===================================================================
file_path = 'final_부동산.csv'
try:
    df = pd.read_csv(file_path)
    print(f":흰색_확인_표시: '{file_path}' 파일 로드 성공!")
except FileNotFoundError:
    print(f":x: 오류: '{file_path}' 경로에 파일이 없습니다. 파일을 먼저 업로드해주세요.")
    exit()

df['기준년'] = df['기준년월']
df['기준월'] = df['기준년월']
# 총자산 = 유동자산 + 비유동자산
df['총자산'] = df['유동자산'] + df['비유동자산']
# 총부채 = 유동부채 + 비유동부채
df['총부채'] = df['유동부채'] + df['비유동부채']
# 부채비율 = 총부채 / 총자산
df['부채비율'] = df['총부채'] / (df['총자산'] + 1e-6)  # 0으로 나누는 것 방지
# 자기자본 = 총자산 - 총부채
df['자기자본'] = df['총자산'] - df['총부채']
# 자기자본비율 = 자기자본 / 총자산
df['자기자본비율'] = df['자기자본'] / (df['총자산'] + 1e-6)
# 총자산회전율 = 매출액 / 총자산
df['총자산회전율'] = df['매출액'] / (df['총자산'] + 1e-6)
# 매출총이익률 = 매출총이익 / 매출액
df['매출총이익률'] = df['매출총이익'] / (df['매출액'] + 1e-6)
# 연체 경험이 있는지 여부 (3년 이내 30일 이상 연체)
df['연체경험여부'] = (
    (df['기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함)'] > 0)
).astype(int)
# 최근 연체 발생 후 경과일수 (음수는 없음으로 처리)
df['연체발생_경과일수'] = df[
    '신용도판단정보공공정보최근발생일자로부터경과일수(CIS)(해제,삭제)'
].clip(lower=0)
# 공공정보 유지건수 존재 여부
df['공공정보_유지여부'] = (
    df['공공정보(국세,지방세,관세체납)건수(CIS)(미해제)'] > 0
).astype(int)
df['총연체대출과목수'] = (
    df['기업신용공여연체과목수(일보)(미해제)']+
    df['기업신용공여연체과목수(일보)(3개월내유지)(해제포함)']+
    df['기업신용공여연체과목수(일보)(6개월내유지)(해제포함)']+
    df['기업신용공여연체과목수(일보)(1년내유지)(해제포함)']+
    df['기업신용공여연체과목수(일보)(3년내유지)(해제포함)']
)
df['총신규연체과목수'] = (
    df['기업신용공여연체과목수(일보)(3개월내발생)(해제포함)']+
    df['기업신용공여연체과목수(일보)(6개월내발생)(해제포함)']+
    df['기업신용공여연체과목수(일보)(1년내발생)(해제포함)']+
    df['기업신용공여연체과목수(일보)(3년내발생)(해제포함)']
)
df['총장기연체과목수'] = (
    df['기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함)']+
    df['기업신용공여연체과목수(일보)(6개월내유지)(연체일수30일이상)(해제포함)']+
    df['기업신용공여연체과목수(일보)(1년내유지)(연체일수30일이상)(해제포함)']+
    df['기업신용공여연체과목수(일보)(3년내유지)(연체일수30일이상)(해제포함)']
)
df['이자원금_30일초과연체과목수'] = (
    df['기업신용공여30일이상연체과목수(일보)(이자연체)(해제포함)']+
    df['기업신용공여30일이상연체과목수(일보)(이자연체)(미해제)']
)
#  नोटबुक에서 확인된 칼럼명 변경 규칙
rename_map = {
    '기업신용공여연체기관수(일보)(3년내유지)(연체일수30일이상)(해제포함)': '연체기관수_3년',
    '기업신용공여연체기관수(일보)(1년내유지)(연체일수30일이상)(해제포함)': '연체기관수_1년',
    '기업신용공여연체기관수(일보)(6개월내유지)(연체일수30일이상)(해제포함)': '연체기관수_6개월',
    '기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함)': '연체기관수_3개월',
    '기업신용공여연체과목수(일보)(3개월내발생)(해제포함)': '연체과목수_3개월발생',
    '기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함)': '연체과목수_3개월유지',
    '기업신용공여30일이상연체기관수(일보)(해제포함)': '연체기관수_전체',
    '기업신용공여연체최장연체일수(일보)(3개월내발생)(해제포함)': '최장연체일수_3개월',
    '기업신용공여연체최장연체일수(일보)(6개월내발생)(해제포함)': '최장연체일수_6개월',
    '기업신용공여연체최장연체일수(일보)(1년내발생)(해제포함)': '최장연체일수_1년',
    '기업신용공여연체최장연체일수(일보)(3년내발생)(해제포함)': '최장연체일수_3년',
    '연체경험여부': '연체경험',
}
df.rename(columns=rename_map, inplace=True)
print(":흰색_확인_표시:  नोटबुक 기반으로 피처명 변경 완료!")


#  नोटबुक에서 확인된 초기 피처 리스트
initial_features = [
    '연체과목수_3개월유지', '연체기관수_전체', '최장연체일수_3개월', '최장연체일수_6개월', '최장연체일수_1년',
    '최장연체일수_3년', '연체경험', '유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계',
    '매출액', '매출총이익', '영업손익', '당기순이익', '영업활동현금흐름', '재무비율_부채비율', '재무비율_유동비율',
    '재무비율_자기자본비율', '재무비율_영업이익율', '재무비율_자기자본이익률(ROE)', 'EBITDA마진율',
    '영업이익증가율', '당기순이익증가율', 'EBITDA증가율', '설립일자', '사업장소유여부', '소유건축물건수',
    '소유건축물권리침해여부', '기업신용평가등급(구간화)', '공공정보_유지여부',
]
#  नोटबुक에서 확인된 타겟 변수
TARGET_COLUMN = '모형개발용Performance(향후1년내부도여부)'
# 결측치 제거
df_clean = df.dropna(subset=initial_features + [TARGET_COLUMN])
# 피처와 타겟 분리
X = df_clean[initial_features]
y = df_clean[TARGET_COLUMN]
# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# ===================================================================
# :셋: 단계: 파이프라인 및 후진 제거법 함수 정의
# ===================================================================
class BankruptcyPredictionPipeline:
    """기업부도 예측 파이프라인"""
    def __init__(self):
        self.model = None
        self.feature_names = None
        self.is_fitted = False
        self.selected_features = []
    def fit(self, X: pd.DataFrame, y: pd.Series):
        self.feature_names = self.selected_features
        X_selected = X[self.feature_names]
        estimators = [
            ('xgb', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')),
            ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))
        ]
        self.model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(class_weight='balanced'), cv=3, n_jobs=-1)
        self.model.fit(X_selected, y)
        self.is_fitted = True
        return self
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        X_selected = X[self.feature_names].copy()
        return self.model.predict(X_selected)
def run_backward_elimination(X_train, y_train, X_test, y_test,
                             initial_features,
                             recall_threshold=0.79,
                             f1_threshold=0.8):
    """후진 제거법을 수행하여 Recall과 F1을 모두 만족시키는 최적의 피처 조합을 찾습니다."""
    current_features = initial_features.copy()
    print(f":로켓: 후진 제거법 시작... (Recall ≥ {recall_threshold}, F1 ≥ {f1_threshold})")
    print("-" * 60)
    step = 1
    while len(current_features) > 1:
        scores = []
        for feat in current_features:
            temp = [f for f in current_features if f != feat]
            pipe = BankruptcyPredictionPipeline()
            pipe.selected_features = temp
            # stdout 억제
            with contextlib.redirect_stdout(io.StringIO()):
                pipe.fit(X_train, y_train)
            y_pred = pipe.predict(X_test)
            r = recall_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            scores.append({'feature': feat, 'recall': r, 'f1': f1})

        # Recall 기준 가장 높은 후보 추출
        best = max(scores, key=lambda x: x['recall'])
        if best['recall'] >= recall_threshold and best['f1'] >= f1_threshold:
            print(f":오른쪽_화살표: 단계 {step}: '{best['feature']}' 제거 "
                  f"(Recall: {best['recall']:.4f}, F1: {best['f1']:.4f})")
            current_features.remove(best['feature'])
            step += 1
        else:
            print(f"\n:경고: 중단: '{best['feature']}' 제거 시 "
                  f"(Recall: {best['recall']:.4f}, F1: {best['f1']:.4f}) 중 하나라도 임계값 미만.")
            break

    print("-" * 60)
    print(":흰색_확인_표시: 후진 제거법 완료!")
    return current_features
# ===================================================================
# :넷: 단계: 실행 및 결과 출력
# ===================================================================
# 사용 가능한 피처만으로 후진 제거법 실행
available_initial_features = [f for f in initial_features if f in X_train.columns]
final_features = run_backward_elimination(X_train, y_train, X_test, y_test, available_initial_features)
print("\n\n--- 최종 결과 ---")
print(f"\n:막대_차트: 최종 선택된 피처 개수: {len(final_features)}")
print("\n:클립보드: 최종 피처 목록:")
for feature in final_features:
    print(f"  - {feature}")

Defaulting to user installation because normal site-packages is not writeable
:흰색_확인_표시: 'final_부동산.csv' 파일 로드 성공!
:흰색_확인_표시:  नोटबुक 기반으로 피처명 변경 완료!
:로켓: 후진 제거법 시작... (Recall ≥ 0.79, F1 ≥ 0.8)
------------------------------------------------------------

:경고: 중단: '당기순이익' 제거 시 (Recall: 0.8610, F1: 0.5826) 중 하나라도 임계값 미만.
------------------------------------------------------------
:흰색_확인_표시: 후진 제거법 완료!


--- 최종 결과 ---

:막대_차트: 최종 선택된 피처 개수: 33

:클립보드: 최종 피처 목록:
  - 연체과목수_3개월유지
  - 연체기관수_전체
  - 최장연체일수_3개월
  - 최장연체일수_6개월
  - 최장연체일수_1년
  - 최장연체일수_3년
  - 연체경험
  - 유동자산
  - 비유동자산
  - 자산총계
  - 유동부채
  - 비유동부채
  - 부채총계
  - 매출액
  - 매출총이익
  - 영업손익
  - 당기순이익
  - 영업활동현금흐름
  - 재무비율_부채비율
  - 재무비율_유동비율
  - 재무비율_자기자본비율
  - 재무비율_영업이익율
  - 재무비율_자기자본이익률(ROE)
  - EBITDA마진율
  - 영업이익증가율
  - 당기순이익증가율
  - EBITDA증가율
  - 설립일자
  - 사업장소유여부
  - 소유건축물건수
  - 소유건축물권리침해여부
  - 기업신용평가등급(구간화)
  - 공공정보_유지여부


In [2]:
# 최종 선택된 피처만 사용하여 모델을 학습하고 테스트셋에 대한 Recall을 계산합니다.

from sklearn.metrics import recall_score,f1_score

# 1) 훈련/테스트 데이터에서 최종 피처만 선택
X_train_selected = X_train[final_features]
X_test_selected  = X_test[final_features]

# 2) 파이프라인 초기화 및 학습
pipeline = BankruptcyPredictionPipeline()
pipeline.selected_features = final_features
pipeline.fit(X_train_selected, y_train)

# 3) 테스트셋 예측 및 Recall 계산
y_pred = pipeline.predict(X_test_selected)
recall = recall_score(y_test, y_pred, pos_label=1)
f1 = f1_score(y_test, y_pred, pos_label=1)

print(f"최종 피처만 사용한 모델의 Recall: {recall:.4f}, F1: {f1:.4f}")

최종 피처만 사용한 모델의 Recall: 0.8468, F1: 0.6082


In [3]:
from sklearn.metrics import confusion_matrix, classification_report

# --- 모델 예측
y_pred = pipeline.predict(X_test_selected)

# --- Confusion Matrix 출력
cm = confusion_matrix(y_test, y_pred)
print("최적화된 모델 결과:")
print("-" * 30)
print("Confusion Matrix:")
print(cm)

# --- Classification Report 출력
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))


최적화된 모델 결과:
------------------------------
Confusion Matrix:
[[24862   722]
 [  118   652]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9953    0.9718    0.9834     25584
           1     0.4745    0.8468    0.6082       770

    accuracy                         0.9681     26354
   macro avg     0.7349    0.9093    0.7958     26354
weighted avg     0.9801    0.9681    0.9724     26354



# 이번에는 recall 성능이 낮아지면 피쳐 제거 X

In [2]:
# ===================================================================
# :일: 단계: 라이브러리 설치 및 임포트
# ===================================================================
!pip install xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import warnings
import contextlib
import io
warnings.filterwarnings('ignore')
# ===================================================================
# :둘: 단계: 데이터 로드 및 전처리
# ===================================================================
file_path = 'final_부동산.csv'
try:
    df = pd.read_csv(file_path)
    print(f":흰색_확인_표시: '{file_path}' 파일 로드 성공!")
except FileNotFoundError:
    print(f":x: 오류: '{file_path}' 경로에 파일이 없습니다. 파일을 먼저 업로드해주세요.")
    exit()

df['기준년'] = df['기준년월']
df['기준월'] = df['기준년월']
# 총자산 = 유동자산 + 비유동자산
df['총자산'] = df['유동자산'] + df['비유동자산']
# 총부채 = 유동부채 + 비유동부채
df['총부채'] = df['유동부채'] + df['비유동부채']
# 부채비율 = 총부채 / 총자산
df['부채비율'] = df['총부채'] / (df['총자산'] + 1e-6)  # 0으로 나누는 것 방지
# 자기자본 = 총자산 - 총부채
df['자기자본'] = df['총자산'] - df['총부채']
# 자기자본비율 = 자기자본 / 총자산
df['자기자본비율'] = df['자기자본'] / (df['총자산'] + 1e-6)
# 총자산회전율 = 매출액 / 총자산
df['총자산회전율'] = df['매출액'] / (df['총자산'] + 1e-6)
# 매출총이익률 = 매출총이익 / 매출액
df['매출총이익률'] = df['매출총이익'] / (df['매출액'] + 1e-6)
# 연체 경험이 있는지 여부 (3년 이내 30일 이상 연체)
df['연체경험여부'] = (
    (df['기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함)'] > 0)
).astype(int)
# 최근 연체 발생 후 경과일수 (음수는 없음으로 처리)
df['연체발생_경과일수'] = df[
    '신용도판단정보공공정보최근발생일자로부터경과일수(CIS)(해제,삭제)'
].clip(lower=0)
# 공공정보 유지건수 존재 여부
df['공공정보_유지여부'] = (
    df['공공정보(국세,지방세,관세체납)건수(CIS)(미해제)'] > 0
).astype(int)
df['총연체대출과목수'] = (
    df['기업신용공여연체과목수(일보)(미해제)']+
    df['기업신용공여연체과목수(일보)(3개월내유지)(해제포함)']+
    df['기업신용공여연체과목수(일보)(6개월내유지)(해제포함)']+
    df['기업신용공여연체과목수(일보)(1년내유지)(해제포함)']+
    df['기업신용공여연체과목수(일보)(3년내유지)(해제포함)']
)
df['총신규연체과목수'] = (
    df['기업신용공여연체과목수(일보)(3개월내발생)(해제포함)']+
    df['기업신용공여연체과목수(일보)(6개월내발생)(해제포함)']+
    df['기업신용공여연체과목수(일보)(1년내발생)(해제포함)']+
    df['기업신용공여연체과목수(일보)(3년내발생)(해제포함)']
)
df['총장기연체과목수'] = (
    df['기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함)']+
    df['기업신용공여연체과목수(일보)(6개월내유지)(연체일수30일이상)(해제포함)']+
    df['기업신용공여연체과목수(일보)(1년내유지)(연체일수30일이상)(해제포함)']+
    df['기업신용공여연체과목수(일보)(3년내유지)(연체일수30일이상)(해제포함)']
)
df['이자원금_30일초과연체과목수'] = (
    df['기업신용공여30일이상연체과목수(일보)(이자연체)(해제포함)']+
    df['기업신용공여30일이상연체과목수(일보)(이자연체)(미해제)']
)
#  नोटबुक에서 확인된 칼럼명 변경 규칙
rename_map = {
    '기업신용공여연체기관수(일보)(3년내유지)(연체일수30일이상)(해제포함)': '연체기관수_3년',
    '기업신용공여연체기관수(일보)(1년내유지)(연체일수30일이상)(해제포함)': '연체기관수_1년',
    '기업신용공여연체기관수(일보)(6개월내유지)(연체일수30일이상)(해제포함)': '연체기관수_6개월',
    '기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함)': '연체기관수_3개월',
    '기업신용공여연체과목수(일보)(3개월내발생)(해제포함)': '연체과목수_3개월발생',
    '기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함)': '연체과목수_3개월유지',
    '기업신용공여30일이상연체기관수(일보)(해제포함)': '연체기관수_전체',
    '기업신용공여연체최장연체일수(일보)(3개월내발생)(해제포함)': '최장연체일수_3개월',
    '기업신용공여연체최장연체일수(일보)(6개월내발생)(해제포함)': '최장연체일수_6개월',
    '기업신용공여연체최장연체일수(일보)(1년내발생)(해제포함)': '최장연체일수_1년',
    '기업신용공여연체최장연체일수(일보)(3년내발생)(해제포함)': '최장연체일수_3년',
    '연체경험여부': '연체경험',
}
df.rename(columns=rename_map, inplace=True)
print(":흰색_확인_표시:  नोटबुक 기반으로 피처명 변경 완료!")


#  नोटबुक에서 확인된 초기 피처 리스트
initial_features = [
    '연체과목수_3개월유지', '연체기관수_전체', '최장연체일수_3개월', '최장연체일수_6개월', '최장연체일수_1년',
    '최장연체일수_3년', '연체경험', '유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계',
    '매출액', '매출총이익', '영업손익', '당기순이익', '영업활동현금흐름', '재무비율_부채비율', '재무비율_유동비율',
    '재무비율_자기자본비율', '재무비율_영업이익율', '재무비율_자기자본이익률(ROE)', 'EBITDA마진율',
    '영업이익증가율', '당기순이익증가율', 'EBITDA증가율', '설립일자', '사업장소유여부', '소유건축물건수',
    '소유건축물권리침해여부', '기업신용평가등급(구간화)', '공공정보_유지여부',
]
#  नोटबुक에서 확인된 타겟 변수
TARGET_COLUMN = '모형개발용Performance(향후1년내부도여부)'
# 결측치 제거
df_clean = df.dropna(subset=initial_features + [TARGET_COLUMN])
# 피처와 타겟 분리
X = df_clean[initial_features]
y = df_clean[TARGET_COLUMN]
# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# ===================================================================
# :셋: 단계: 파이프라인 및 후진 제거법 함수 정의
# ===================================================================
class BankruptcyPredictionPipeline:
    """기업부도 예측 파이프라인"""
    def __init__(self):
        self.model = None
        self.feature_names = None
        self.is_fitted = False
        self.selected_features = []
    def fit(self, X: pd.DataFrame, y: pd.Series):
        self.feature_names = self.selected_features
        X_selected = X[self.feature_names]
        estimators = [
            ('xgb', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')),
            ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))
        ]
        self.model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(class_weight='balanced'), cv=3, n_jobs=-1)
        self.model.fit(X_selected, y)
        self.is_fitted = True
        return self
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        X_selected = X[self.feature_names].copy()
        return self.model.predict(X_selected)
def run_improved_backward_elimination(X_train, y_train, X_test, y_test, initial_features, recall_threshold=0.79):
    """조건부 후진 제거법 (각 피처 평가 후 제거할지 말지 결정하며 계속 진행)"""
    current_features = initial_features.copy()
    previous_best_recall = 0.0
    print(f":로켓: 개선된 조건부 후진 제거법 시작... (Recall 기준: {recall_threshold} 이상)")
    print("-" * 60)
    step = 1

    for feature in initial_features:
        if feature not in current_features:
            continue  # 이미 제거된 피처면 건너뜀

        # 해당 feature를 제외한 임시 feature set
        temp_features = [f for f in current_features if f != feature]
        pipeline = BankruptcyPredictionPipeline()
        pipeline.selected_features = temp_features

        # 학습 및 평가
        with contextlib.redirect_stdout(io.StringIO()):
            pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        recall = recall_score(y_test, y_pred, pos_label=1)

        # 제거한 후 recall이 더 좋거나 같으면 제거
        if recall >= recall_threshold and recall >= previous_best_recall:
            print(f":오른쪽_화살표: 단계 {step}: '{feature}' 제거 (Recall: {recall:.4f})")
            current_features.remove(feature)
            previous_best_recall = recall
            step += 1
        else:
            print(f":경고: '{feature}' 제거 시 Recall이 {recall:.4f}로 낮아짐 → 유지")

    print("-" * 60)
    print(":흰색_확인_표시: 개선된 후진 제거법 완료!")
    return current_features

# ===================================================================
# :넷: 단계: 실행 및 결과 출력
# ===================================================================
# 사용 가능한 피처만으로 후진 제거법 실행
available_initial_features = [f for f in initial_features if f in X_train.columns]
final_features = run_improved_backward_elimination(X_train, y_train, X_test, y_test, available_initial_features)
print("\n\n--- 최종 결과 ---")
print(f"\n:막대_차트: 최종 선택된 피처 개수: {len(final_features)}")
print("\n:클립보드: 최종 피처 목록:")
for feature in final_features:
    print(f"  - {feature}")

Defaulting to user installation because normal site-packages is not writeable
:흰색_확인_표시: 'final_부동산.csv' 파일 로드 성공!
:흰색_확인_표시:  नोटबुक 기반으로 피처명 변경 완료!
:로켓: 개선된 조건부 후진 제거법 시작... (Recall 기준: 0.79 이상)
------------------------------------------------------------
:오른쪽_화살표: 단계 1: '연체과목수_3개월유지' 제거 (Recall: 0.8403)
:오른쪽_화살표: 단계 2: '연체기관수_전체' 제거 (Recall: 0.8455)
:경고: '최장연체일수_3개월' 제거 시 Recall이 0.8442로 낮아짐 → 유지
:오른쪽_화살표: 단계 3: '최장연체일수_6개월' 제거 (Recall: 0.8494)
:오른쪽_화살표: 단계 4: '최장연체일수_1년' 제거 (Recall: 0.8519)
:오른쪽_화살표: 단계 5: '최장연체일수_3년' 제거 (Recall: 0.8610)
:경고: '연체경험' 제거 시 Recall이 0.8571로 낮아짐 → 유지
:경고: '유동자산' 제거 시 Recall이 0.8481로 낮아짐 → 유지
:경고: '비유동자산' 제거 시 Recall이 0.8494로 낮아짐 → 유지
:경고: '자산총계' 제거 시 Recall이 0.8558로 낮아짐 → 유지
:오른쪽_화살표: 단계 6: '유동부채' 제거 (Recall: 0.8636)
:경고: '비유동부채' 제거 시 Recall이 0.8558로 낮아짐 → 유지
:경고: '부채총계' 제거 시 Recall이 0.8519로 낮아짐 → 유지
:경고: '매출액' 제거 시 Recall이 0.8442로 낮아짐 → 유지
:경고: '매출총이익' 제거 시 Recall이 0.8623로 낮아짐 → 유지
:경고: '영업손익' 제거 시 Recall이 0.8455로 낮아짐 → 유지
:경고: '당기순이익' 제거 시 Recall이 0.8

# 다시 recall만

In [4]:
# ===================================================================
# :일: 단계: 라이브러리 설치 및 임포트
# ===================================================================
!pip install xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import warnings
import contextlib
import io
warnings.filterwarnings('ignore')
# ===================================================================
# :둘: 단계: 데이터 로드 및 전처리
# ===================================================================
file_path = 'final_부동산.csv'
try:
    df = pd.read_csv(file_path)
    print(f":흰색_확인_표시: '{file_path}' 파일 로드 성공!")
except FileNotFoundError:
    print(f":x: 오류: '{file_path}' 경로에 파일이 없습니다. 파일을 먼저 업로드해주세요.")
    exit()

df['기준년'] = df['기준년월']
df['기준월'] = df['기준년월']
# 총자산 = 유동자산 + 비유동자산
df['총자산'] = df['유동자산'] + df['비유동자산']
# 총부채 = 유동부채 + 비유동부채
df['총부채'] = df['유동부채'] + df['비유동부채']
# 부채비율 = 총부채 / 총자산
df['부채비율'] = df['총부채'] / (df['총자산'] + 1e-6)  # 0으로 나누는 것 방지
# 자기자본 = 총자산 - 총부채
df['자기자본'] = df['총자산'] - df['총부채']
# 자기자본비율 = 자기자본 / 총자산
df['자기자본비율'] = df['자기자본'] / (df['총자산'] + 1e-6)
# 총자산회전율 = 매출액 / 총자산
df['총자산회전율'] = df['매출액'] / (df['총자산'] + 1e-6)
# 매출총이익률 = 매출총이익 / 매출액
df['매출총이익률'] = df['매출총이익'] / (df['매출액'] + 1e-6)
# 연체 경험이 있는지 여부 (3년 이내 30일 이상 연체)
df['연체경험여부'] = (
    (df['기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함)'] > 0)
).astype(int)
# 최근 연체 발생 후 경과일수 (음수는 없음으로 처리)
df['연체발생_경과일수'] = df[
    '신용도판단정보공공정보최근발생일자로부터경과일수(CIS)(해제,삭제)'
].clip(lower=0)
# 공공정보 유지건수 존재 여부
df['공공정보_유지여부'] = (
    df['공공정보(국세,지방세,관세체납)건수(CIS)(미해제)'] > 0
).astype(int)
df['총연체대출과목수'] = (
    df['기업신용공여연체과목수(일보)(미해제)']+
    df['기업신용공여연체과목수(일보)(3개월내유지)(해제포함)']+
    df['기업신용공여연체과목수(일보)(6개월내유지)(해제포함)']+
    df['기업신용공여연체과목수(일보)(1년내유지)(해제포함)']+
    df['기업신용공여연체과목수(일보)(3년내유지)(해제포함)']
)
df['총신규연체과목수'] = (
    df['기업신용공여연체과목수(일보)(3개월내발생)(해제포함)']+
    df['기업신용공여연체과목수(일보)(6개월내발생)(해제포함)']+
    df['기업신용공여연체과목수(일보)(1년내발생)(해제포함)']+
    df['기업신용공여연체과목수(일보)(3년내발생)(해제포함)']
)
df['총장기연체과목수'] = (
    df['기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함)']+
    df['기업신용공여연체과목수(일보)(6개월내유지)(연체일수30일이상)(해제포함)']+
    df['기업신용공여연체과목수(일보)(1년내유지)(연체일수30일이상)(해제포함)']+
    df['기업신용공여연체과목수(일보)(3년내유지)(연체일수30일이상)(해제포함)']
)
df['이자원금_30일초과연체과목수'] = (
    df['기업신용공여30일이상연체과목수(일보)(이자연체)(해제포함)']+
    df['기업신용공여30일이상연체과목수(일보)(이자연체)(미해제)']
)
#  नोटबुक에서 확인된 칼럼명 변경 규칙
rename_map = {
    '기업신용공여연체기관수(일보)(3년내유지)(연체일수30일이상)(해제포함)': '연체기관수_3년',
    '기업신용공여연체기관수(일보)(1년내유지)(연체일수30일이상)(해제포함)': '연체기관수_1년',
    '기업신용공여연체기관수(일보)(6개월내유지)(연체일수30일이상)(해제포함)': '연체기관수_6개월',
    '기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함)': '연체기관수_3개월',
    '기업신용공여연체과목수(일보)(3개월내발생)(해제포함)': '연체과목수_3개월발생',
    '기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함)': '연체과목수_3개월유지',
    '기업신용공여30일이상연체기관수(일보)(해제포함)': '연체기관수_전체',
    '기업신용공여연체최장연체일수(일보)(3개월내발생)(해제포함)': '최장연체일수_3개월',
    '기업신용공여연체최장연체일수(일보)(6개월내발생)(해제포함)': '최장연체일수_6개월',
    '기업신용공여연체최장연체일수(일보)(1년내발생)(해제포함)': '최장연체일수_1년',
    '기업신용공여연체최장연체일수(일보)(3년내발생)(해제포함)': '최장연체일수_3년',
    '연체경험여부': '연체경험',
}
df.rename(columns=rename_map, inplace=True)
print(":흰색_확인_표시:  नोटबुक 기반으로 피처명 변경 완료!")


#  नोटबुक에서 확인된 초기 피처 리스트
initial_features = [
    '연체과목수_3개월유지', '연체기관수_전체', '최장연체일수_3개월', '최장연체일수_6개월', '최장연체일수_1년',
    '최장연체일수_3년', '연체경험', '유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계',
    '매출액', '매출총이익', '영업손익', '당기순이익', '영업활동현금흐름', '재무비율_부채비율', '재무비율_유동비율',
    '재무비율_자기자본비율', '재무비율_영업이익율', '재무비율_자기자본이익률(ROE)', 'EBITDA마진율',
    '영업이익증가율', '당기순이익증가율', 'EBITDA증가율', '설립일자', '사업장소유여부', '소유건축물건수',
    '소유건축물권리침해여부', '기업신용평가등급(구간화)', '공공정보_유지여부',
]
#  नोटबुक에서 확인된 타겟 변수
TARGET_COLUMN = '모형개발용Performance(향후1년내부도여부)'
# 결측치 제거
df_clean = df.dropna(subset=initial_features + [TARGET_COLUMN])
# 피처와 타겟 분리
X = df_clean[initial_features]
y = df_clean[TARGET_COLUMN]
# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# ===================================================================
# :셋: 단계: 파이프라인 및 후진 제거법 함수 정의
# ===================================================================
class BankruptcyPredictionPipeline:
    """기업부도 예측 파이프라인"""
    def __init__(self):
        self.model = None
        self.feature_names = None
        self.is_fitted = False
        self.selected_features = []
    def fit(self, X: pd.DataFrame, y: pd.Series):
        self.feature_names = self.selected_features
        X_selected = X[self.feature_names]
        estimators = [
            ('xgb', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')),
            ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))
        ]
        self.model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(class_weight='balanced'), cv=3, n_jobs=-1)
        self.model.fit(X_selected, y)
        self.is_fitted = True
        return self
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        X_selected = X[self.feature_names].copy()
        return self.model.predict(X_selected)
def run_backward_elimination(X_train, y_train, X_test, y_test, initial_features, recall_threshold=0.79):
    """후진 제거법을 수행하여 최적의 피처 조합을 찾습니다."""
    current_features = initial_features.copy()
    print(f":로켓: 후진 제거법 시작... (Recall 유지 조건: {recall_threshold} 이상)")
    print("-" * 60)
    step = 1
    while len(current_features) > 1:
        recall_per_feature = []
        for feature_to_remove in current_features:
            temp_features = [f for f in current_features if f != feature_to_remove]
            pipeline = BankruptcyPredictionPipeline()
            pipeline.selected_features = temp_features
            with contextlib.redirect_stdout(io.StringIO()):
                pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)
            recall = recall_score(y_test, y_pred, pos_label=1)
            recall_per_feature.append({'feature': feature_to_remove, 'recall': recall})
        best_candidate = max(recall_per_feature, key=lambda x: x['recall'])
        if best_candidate['recall'] >= recall_threshold:
            feature_to_eliminate = best_candidate['feature']
            recall_at_elimination = best_candidate['recall']
            print(f":오른쪽_화살표: 단계 {step}: '{feature_to_eliminate}' 제거 (제거 후 Recall: {recall_at_elimination:.4f})")
            current_features.remove(feature_to_eliminate)
            step += 1
        else:
            print(f"\n:경고: 중단: '{best_candidate['feature']}' 제거 시 Recall({best_candidate['recall']:.4f})이 임계값 미만.")
            break
    print("-" * 60)
    print(":흰색_확인_표시: 후진 제거법 완료!")
    return current_features
# ===================================================================
# :넷: 단계: 실행 및 결과 출력
# ===================================================================
# 사용 가능한 피처만으로 후진 제거법 실행
available_initial_features = [f for f in initial_features if f in X_train.columns]
final_features = run_backward_elimination(X_train, y_train, X_test, y_test, available_initial_features)
print("\n\n--- 최종 결과 ---")
print(f"\n:막대_차트: 최종 선택된 피처 개수: {len(final_features)}")
print("\n:클립보드: 최종 피처 목록:")
for feature in final_features:
    print(f"  - {feature}")

Defaulting to user installation because normal site-packages is not writeable
:흰색_확인_표시: 'final_부동산.csv' 파일 로드 성공!
:흰색_확인_표시:  नोटबुक 기반으로 피처명 변경 완료!
:로켓: 후진 제거법 시작... (Recall 유지 조건: 0.79 이상)
------------------------------------------------------------
:오른쪽_화살표: 단계 1: '당기순이익' 제거 (제거 후 Recall: 0.8610)
:오른쪽_화살표: 단계 2: '공공정보_유지여부' 제거 (제거 후 Recall: 0.8636)
:오른쪽_화살표: 단계 3: '유동자산' 제거 (제거 후 Recall: 0.8623)
:오른쪽_화살표: 단계 4: '연체기관수_전체' 제거 (제거 후 Recall: 0.8636)
:오른쪽_화살표: 단계 5: '최장연체일수_6개월' 제거 (제거 후 Recall: 0.8610)
:오른쪽_화살표: 단계 6: '최장연체일수_3년' 제거 (제거 후 Recall: 0.8597)
:오른쪽_화살표: 단계 7: '영업손익' 제거 (제거 후 Recall: 0.8636)
:오른쪽_화살표: 단계 8: '비유동자산' 제거 (제거 후 Recall: 0.8662)
:오른쪽_화살표: 단계 9: '최장연체일수_3개월' 제거 (제거 후 Recall: 0.8636)
:오른쪽_화살표: 단계 10: '재무비율_유동비율' 제거 (제거 후 Recall: 0.8623)
:오른쪽_화살표: 단계 11: '설립일자' 제거 (제거 후 Recall: 0.8636)
:오른쪽_화살표: 단계 12: '재무비율_부채비율' 제거 (제거 후 Recall: 0.8610)
:오른쪽_화살표: 단계 13: '자산총계' 제거 (제거 후 Recall: 0.8675)
:오른쪽_화살표: 단계 14: '최장연체일수_1년' 제거 (제거 후 Recall: 0.8649)
:오른쪽_화살표: 단계 15: '유동부채' 제거 

In [6]:
# 최종 선택된 피처만 사용하여 모델을 학습하고 테스트셋에 대한 Recall을 계산합니다.

from sklearn.metrics import recall_score,f1_score
from sklearn.metrics import confusion_matrix, classification_report

# 1) 훈련/테스트 데이터에서 최종 피처만 선택
X_train_selected = X_train[final_features]
X_test_selected  = X_test[final_features]

# 2) 파이프라인 초기화 및 학습
pipeline = BankruptcyPredictionPipeline()
pipeline.selected_features = final_features
pipeline.fit(X_train_selected, y_train)

# --- 모델 예측
y_pred = pipeline.predict(X_test_selected)

# --- Confusion Matrix 출력
cm = confusion_matrix(y_test, y_pred)
print("최적화된 모델 결과:")
print("-" * 30)
print("Confusion Matrix:")
print(cm)

# --- Classification Report 출력
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))


최적화된 모델 결과:
------------------------------
Confusion Matrix:
[[19010  6574]
 [  132   638]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9931    0.7430    0.8501     25584
           1     0.0885    0.8286    0.1599       770

    accuracy                         0.7455     26354
   macro avg     0.5408    0.7858    0.5050     26354
weighted avg     0.9667    0.7455    0.8299     26354

